# Question 1

In [522]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

In [523]:
df=pd.read_csv('USA_Housing.csv')
df

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price
0,79545.45857,5.682861,7.009188,4.09,23086.80050,1.059034e+06
1,79248.64245,6.002900,6.730821,3.09,40173.07217,1.505891e+06
2,61287.06718,5.865890,8.512727,5.13,36882.15940,1.058988e+06
3,63345.24005,7.188236,5.586729,3.26,34310.24283,1.260617e+06
4,59982.19723,5.040555,7.839388,4.23,26354.10947,6.309435e+05
...,...,...,...,...,...,...
4995,60567.94414,7.830362,6.137356,3.46,22837.36103,1.060194e+06
4996,78491.27543,6.999135,6.576763,4.02,25616.11549,1.482618e+06
4997,63390.68689,7.250591,4.805081,2.13,33266.14549,1.030730e+06
4998,68001.33124,5.534388,7.130144,5.44,42625.62016,1.198657e+06


## (a) Separate X & Y

In [524]:
X=df.iloc[:, 0:5].copy()
X

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population
0,79545.45857,5.682861,7.009188,4.09,23086.80050
1,79248.64245,6.002900,6.730821,3.09,40173.07217
2,61287.06718,5.865890,8.512727,5.13,36882.15940
3,63345.24005,7.188236,5.586729,3.26,34310.24283
4,59982.19723,5.040555,7.839388,4.23,26354.10947
...,...,...,...,...,...
4995,60567.94414,7.830362,6.137356,3.46,22837.36103
4996,78491.27543,6.999135,6.576763,4.02,25616.11549
4997,63390.68689,7.250591,4.805081,2.13,33266.14549
4998,68001.33124,5.534388,7.130144,5.44,42625.62016


In [525]:
Y=df.iloc[:, 5].copy()
Y

0       1.059034e+06
1       1.505891e+06
2       1.058988e+06
3       1.260617e+06
4       6.309435e+05
            ...     
4995    1.060194e+06
4996    1.482618e+06
4997    1.030730e+06
4998    1.198657e+06
4999    1.298950e+06
Name: Price, Length: 5000, dtype: float64

## (b) Scale input features

In [526]:
ss=StandardScaler()
Xstd=ss.fit_transform(X)
X['Avg. Area Income']=Xstd[:, 0]
X['Area Population']=Xstd[:, 4]
X

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population
0,1.028660,5.682861,7.009188,4.09,-1.317599
1,1.000808,6.002900,6.730821,3.09,0.403999
2,-0.684629,5.865890,8.512727,5.13,0.072410
3,-0.491499,7.188236,5.586729,3.26,-0.186734
4,-0.807073,5.040555,7.839388,4.23,-0.988387
...,...,...,...,...,...
4995,-0.752109,7.830362,6.137356,3.46,-1.342732
4996,0.929740,6.999135,6.576763,4.02,-1.062747
4997,-0.487235,7.250591,4.805081,2.13,-0.291937
4998,-0.054592,5.534388,7.130144,5.44,0.651116


## (c) & (d)

In [527]:
kf=KFold(n_splits=5, random_state=None)

Beta_collection=[]
R2_collection=[]

for train_index , test_index in kf.split(X):
    X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]
    Y_train , Y_test = Y[train_index] , Y[test_index]


    #Prepend X_test with a column of 1s
    no_of_rows, no_of_columns=X_test.shape
    X_ones=np.ones((no_of_rows, 1))
    X_test_with_ones=np.hstack((X_ones, X_test))


    #Prepend X_train with a column of 1s
    no_of_rows, no_of_columns=X_train.shape
    X_ones=np.ones((no_of_rows, 1))
    X_train_with_ones=np.hstack((X_ones, X_train))

    #Least Squares Method
    X_transpose=np.transpose(X_train_with_ones)
    X_transpose_X=np.matmul(X_transpose,X_train_with_ones)
    X_transpose_X_inverse=np.linalg.inv(X_transpose_X)
    X_transpose_X_inverse_X=np.matmul( X_transpose_X_inverse, X_transpose)
    B=np.matmul( X_transpose_X_inverse_X,Y_train)

    Y_predicted=np.matmul(X_test_with_ones,B)

    score=r2_score(Y_test, Y_predicted)

    #Store all beta matrices and R2 scores
    Beta_collection.append(B)
    R2_collection.append(score)

    

## (e)

In [528]:
#Select Beta with max R2 score
R2_max_index=R2_collection.index(max(R2_collection))
B=Beta_collection[R2_max_index]
print(f"Best R2 Score={max(R2_collection)}")
print(f"Corresponding beta matrix:\n{B}")

Best R2 Score=0.920850383697766
Corresponding beta matrix:
[-596076.10240762  229921.55808488  165957.41875628  119055.00734636
     911.05670455  151317.80239611]


In [529]:
#Test for 30% data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 0)


#Prepend X_test with a column of 1s
no_of_rows, no_of_columns=X_test.shape
X_ones=np.ones((no_of_rows, 1))
X_test_with_ones=np.hstack((X_ones, X_test))

Y_predicted=np.matmul(X_test_with_ones,B)
score=r2_score(Y_test, Y_predicted)

print(f"For 30% of data, R2 Score={score}")

For 30% of data, R2 Score=0.9200994781500813


# Question 2

In [530]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

In [531]:
df=pd.read_csv('USA_Housing.csv')
df

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price
0,79545.45857,5.682861,7.009188,4.09,23086.80050,1.059034e+06
1,79248.64245,6.002900,6.730821,3.09,40173.07217,1.505891e+06
2,61287.06718,5.865890,8.512727,5.13,36882.15940,1.058988e+06
3,63345.24005,7.188236,5.586729,3.26,34310.24283,1.260617e+06
4,59982.19723,5.040555,7.839388,4.23,26354.10947,6.309435e+05
...,...,...,...,...,...,...
4995,60567.94414,7.830362,6.137356,3.46,22837.36103,1.060194e+06
4996,78491.27543,6.999135,6.576763,4.02,25616.11549,1.482618e+06
4997,63390.68689,7.250591,4.805081,2.13,33266.14549,1.030730e+06
4998,68001.33124,5.534388,7.130144,5.44,42625.62016,1.198657e+06


## Separate X & Y

In [532]:
X=df.iloc[:, 0:5].copy()
X

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population
0,79545.45857,5.682861,7.009188,4.09,23086.80050
1,79248.64245,6.002900,6.730821,3.09,40173.07217
2,61287.06718,5.865890,8.512727,5.13,36882.15940
3,63345.24005,7.188236,5.586729,3.26,34310.24283
4,59982.19723,5.040555,7.839388,4.23,26354.10947
...,...,...,...,...,...
4995,60567.94414,7.830362,6.137356,3.46,22837.36103
4996,78491.27543,6.999135,6.576763,4.02,25616.11549
4997,63390.68689,7.250591,4.805081,2.13,33266.14549
4998,68001.33124,5.534388,7.130144,5.44,42625.62016


In [533]:
Y=df.iloc[:, 5].copy()
Y

0       1.059034e+06
1       1.505891e+06
2       1.058988e+06
3       1.260617e+06
4       6.309435e+05
            ...     
4995    1.060194e+06
4996    1.482618e+06
4997    1.030730e+06
4998    1.198657e+06
4999    1.298950e+06
Name: Price, Length: 5000, dtype: float64

## Scale input features

In [534]:
ss=StandardScaler()
Xstd=ss.fit_transform(X)
X['Avg. Area Income']=Xstd[:, 0]
X['Area Population']=Xstd[:, 4]
X

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population
0,1.028660,5.682861,7.009188,4.09,-1.317599
1,1.000808,6.002900,6.730821,3.09,0.403999
2,-0.684629,5.865890,8.512727,5.13,0.072410
3,-0.491499,7.188236,5.586729,3.26,-0.186734
4,-0.807073,5.040555,7.839388,4.23,-0.988387
...,...,...,...,...,...
4995,-0.752109,7.830362,6.137356,3.46,-1.342732
4996,0.929740,6.999135,6.576763,4.02,-1.062747
4997,-0.487235,7.250591,4.805081,2.13,-0.291937
4998,-0.054592,5.534388,7.130144,5.44,0.651116


## Model Training

In [535]:
def GradientDescent(alpha):
    
    X_train, X_val, X_test=np.split(X, [int(.56 * len(X)), int(.7 * len(X))])
    Y_train, Y_val, Y_test=np.split(Y, [int(.56 * len(X)), int(.7 * len(Y))])

    no_of_iterations=1000
    
    #Prepend X_val with a column of 1s
    no_of_rows, no_of_columns=X_val.shape
    X_ones=np.ones((no_of_rows, 1))
    X_val_with_ones=np.hstack((X_ones, X_val))

    #Prepend X_test with a column of 1s
    no_of_rows, no_of_columns=X_test.shape
    X_ones=np.ones((no_of_rows, 1))
    X_test_with_ones=np.hstack((X_ones, X_test))

    #Prepend X_train with a column of 1s
    no_of_rows, no_of_columns=X_train.shape
    X_ones=np.ones((no_of_rows, 1))
    X_train_with_ones=np.hstack((X_ones, X_train))


    B=[0]*(no_of_columns+1)
    for iteration in range(0, no_of_iterations):
        
        new_B=[]
        for beta_index in range(0, len(B)):
            
            cost=0
            sum=0
            for row_index in range(0, no_of_rows):
                
                BX_sum=0
                for i in range(0, len(B)):
                    BX_sum=BX_sum + B[i]*X_train_with_ones[row_index][i]                            
                    
                sum = sum + (BX_sum - Y[row_index]) * X_train_with_ones[row_index][beta_index]   

            new_B_val = B[beta_index] - (alpha/no_of_rows)*sum
            new_B.append(new_B_val)

        
        B=new_B

    Y_val_predicted=np.matmul(X_val_with_ones, B)
    Y_predicted=np.matmul(X_test_with_ones, B)

    print(f"alpha={alpha}\t\tValidation set R2 score={r2_score(Y_val, Y_val_predicted)}\t\tTest set R2 score={r2_score(Y_test, Y_predicted)}")

    return B


In [536]:
B1=GradientDescent(1e-3)

alpha=0.001		Validation set R2 score=0.7965207895591857		Test set R2 score=0.7978318643275376


In [537]:
B2=GradientDescent(1e-2)

alpha=0.01		Validation set R2 score=0.8797011651448852		Test set R2 score=0.8871567031437755


In [538]:
B3=GradientDescent(1e-1)

/tmp/ipykernel_10502/3310583464.py:38: RuntimeWarning: overflow encountered in double_scalars
  sum = sum + (BX_sum - Y[row_index]) * X_train_with_ones[row_index][beta_index]
/tmp/ipykernel_10502/3310583464.py:38: RuntimeWarning: invalid value encountered in double_scalars
  sum = sum + (BX_sum - Y[row_index]) * X_train_with_ones[row_index][beta_index]
/tmp/ipykernel_10502/3310583464.py:40: RuntimeWarning: invalid value encountered in double_scalars
  new_B_val = B[beta_index] - (alpha/no_of_rows)*sum


ValueError: Input contains NaN.

In [ ]:
B4=GradientDescent(1)

/tmp/ipykernel_5590/3310583464.py:38: RuntimeWarning: overflow encountered in double_scalars
  sum = sum + (BX_sum - Y[row_index]) * X_train_with_ones[row_index][beta_index]
/tmp/ipykernel_5590/3310583464.py:40: RuntimeWarning: invalid value encountered in double_scalars
  new_B_val = B[beta_index] - (alpha/no_of_rows)*sum
/tmp/ipykernel_5590/3310583464.py:38: RuntimeWarning: invalid value encountered in double_scalars
  sum = sum + (BX_sum - Y[row_index]) * X_train_with_ones[row_index][beta_index]


ValueError: Input contains NaN.

### Alpha>0.01 gives too high of a R2 score => Error

### Best R2 score is with alpha=0.01

In [539]:
print(f"Best beta coefficients are={B2}")

Best beta coefficients are=[-50238.29172489715, 231070.35218036757, 128142.41526378704, 73896.83897392264, 1708.070304257673, 149343.32069860815]


# Question 3

In [540]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA



## 1.

In [541]:
df=pd.read_csv("imports-85.data")
df.columns=["symboling", "normalized_losses",
"make", "fuel_type", "aspiration","num_doors", "body_style", "drive_wheels",
"engine_location", "wheel_base", "length", "width", "height", "curb_weight",
"engine_type", "num_cylinders", "engine_size", "fuel_system", "bore", "stroke",
"compression_ratio", "horsepower", "peak_rpm", "city_mpg", "highway_mpg", "price"]
df


,symboling,normalized_losses,make,fuel_type,aspiration,num_doors,body_style,drive_wheels,engine_location,wheel_base,...,engine_size,fuel_system,bore,stroke,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
1,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
2,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
3,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450
4,2,?,audi,gas,std,two,sedan,fwd,front,99.8,...,136,mpfi,3.19,3.40,8.5,110,5500,19,25,15250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,-1,95,volvo,gas,std,four,sedan,rwd,front,109.1,...,141,mpfi,3.78,3.15,9.5,114,5400,23,28,16845
200,-1,95,volvo,gas,turbo,four,sedan,rwd,front,109.1,...,141,mpfi,3.78,3.15,8.7,160,5300,19,25,19045
201,-1,95,volvo,gas,std,four,sedan,rwd,front,109.1,...,173,mpfi,3.58,2.87,8.8,134,5500,18,23,21485
202,-1,95,volvo,diesel,turbo,four,sedan,rwd,front,109.1,...,145,idi,3.01,3.40,23.0,106,4800,26,27,22470


In [542]:
df=df.replace("?",np.nan)
df

,symboling,normalized_losses,make,fuel_type,aspiration,num_doors,body_style,drive_wheels,engine_location,wheel_base,...,engine_size,fuel_system,bore,stroke,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price
0,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
1,1,NaN,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
2,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
3,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450
4,2,NaN,audi,gas,std,two,sedan,fwd,front,99.8,...,136,mpfi,3.19,3.40,8.5,110,5500,19,25,15250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,-1,95,volvo,gas,std,four,sedan,rwd,front,109.1,...,141,mpfi,3.78,3.15,9.5,114,5400,23,28,16845
200,-1,95,volvo,gas,turbo,four,sedan,rwd,front,109.1,...,141,mpfi,3.78,3.15,8.7,160,5300,19,25,19045
201,-1,95,volvo,gas,std,four,sedan,rwd,front,109.1,...,173,mpfi,3.58,2.87,8.8,134,5500,18,23,21485
202,-1,95,volvo,diesel,turbo,four,sedan,rwd,front,109.1,...,145,idi,3.01,3.40,23.0,106,4800,26,27,22470


## 2.

In [543]:
df.isna().sum()

symboling             0
normalized_losses    40
make                  0
fuel_type             0
aspiration            0
num_doors             2
body_style            0
drive_wheels          0
engine_location       0
wheel_base            0
length                0
width                 0
height                0
curb_weight           0
engine_type           0
num_cylinders         0
engine_size           0
fuel_system           0
bore                  4
stroke                4
compression_ratio     0
horsepower            2
peak_rpm              2
city_mpg              0
highway_mpg           0
price                 4
dtype: int64

In [544]:
mean_imputer=SimpleImputer(missing_values=np.nan,strategy="mean")
df1=pd.DataFrame((mean_imputer.fit_transform((df['normalized_losses'].values).reshape(-1, 1))).flatten(), columns=['normalized_losses'])
df1['bore']=pd.DataFrame((mean_imputer.fit_transform((df['bore'].values).reshape(-1, 1))).flatten())
df1['stroke']=pd.DataFrame((mean_imputer.fit_transform((df['stroke'].values).reshape(-1, 1))).flatten())
df1['horsepower']=pd.DataFrame((mean_imputer.fit_transform((df['horsepower'].values).reshape(-1, 1))).flatten())
df1['peak_rpm']=pd.DataFrame((mean_imputer.fit_transform((df['peak_rpm'].values).reshape(-1, 1))).flatten())

for col in df.columns:
    if col not in ['normalized_losses', 'bore', 'stroke', 'horsepower', 'peak_rpm']:
        df1[col]=df[col]

df=df1
df

,normalized_losses,bore,stroke,horsepower,peak_rpm,symboling,make,fuel_type,aspiration,num_doors,...,height,curb_weight,engine_type,num_cylinders,engine_size,fuel_system,compression_ratio,city_mpg,highway_mpg,price
0,122.0,3.47,2.68,111.0,5000.0,3,alfa-romero,gas,std,two,...,48.8,2548,dohc,four,130,mpfi,9.0,21,27,16500
1,122.0,2.68,3.47,154.0,5000.0,1,alfa-romero,gas,std,two,...,52.4,2823,ohcv,six,152,mpfi,9.0,19,26,16500
2,164.0,3.19,3.40,102.0,5500.0,2,audi,gas,std,four,...,54.3,2337,ohc,four,109,mpfi,10.0,24,30,13950
3,164.0,3.19,3.40,115.0,5500.0,2,audi,gas,std,four,...,54.3,2824,ohc,five,136,mpfi,8.0,18,22,17450
4,122.0,3.19,3.40,110.0,5500.0,2,audi,gas,std,two,...,53.1,2507,ohc,five,136,mpfi,8.5,19,25,15250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,95.0,3.78,3.15,114.0,5400.0,-1,volvo,gas,std,four,...,55.5,2952,ohc,four,141,mpfi,9.5,23,28,16845
200,95.0,3.78,3.15,160.0,5300.0,-1,volvo,gas,turbo,four,...,55.5,3049,ohc,four,141,mpfi,8.7,19,25,19045
201,95.0,3.58,2.87,134.0,5500.0,-1,volvo,gas,std,four,...,55.5,3012,ohcv,six,173,mpfi,8.8,18,23,21485
202,95.0,3.01,3.40,106.0,4800.0,-1,volvo,diesel,turbo,four,...,55.5,3217,ohc,six,145,idi,23.0,26,27,22470


In [545]:
df=df.dropna(subset=['price'])
df.isna().sum()

normalized_losses    0
bore                 0
stroke               0
horsepower           0
peak_rpm             0
symboling            0
make                 0
fuel_type            0
aspiration           0
num_doors            2
body_style           0
drive_wheels         0
engine_location      0
wheel_base           0
length               0
width                0
height               0
curb_weight          0
engine_type          0
num_cylinders        0
engine_size          0
fuel_system          0
compression_ratio    0
city_mpg             0
highway_mpg          0
price                0
dtype: int64

## 3.

In [546]:
df.reset_index(inplace=True)
df=df.drop('index', axis=1)
df

,normalized_losses,bore,stroke,horsepower,peak_rpm,symboling,make,fuel_type,aspiration,num_doors,...,height,curb_weight,engine_type,num_cylinders,engine_size,fuel_system,compression_ratio,city_mpg,highway_mpg,price
0,122.0,3.47,2.68,111.0,5000.0,3,alfa-romero,gas,std,two,...,48.8,2548,dohc,four,130,mpfi,9.0,21,27,16500
1,122.0,2.68,3.47,154.0,5000.0,1,alfa-romero,gas,std,two,...,52.4,2823,ohcv,six,152,mpfi,9.0,19,26,16500
2,164.0,3.19,3.40,102.0,5500.0,2,audi,gas,std,four,...,54.3,2337,ohc,four,109,mpfi,10.0,24,30,13950
3,164.0,3.19,3.40,115.0,5500.0,2,audi,gas,std,four,...,54.3,2824,ohc,five,136,mpfi,8.0,18,22,17450
4,122.0,3.19,3.40,110.0,5500.0,2,audi,gas,std,two,...,53.1,2507,ohc,five,136,mpfi,8.5,19,25,15250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,95.0,3.78,3.15,114.0,5400.0,-1,volvo,gas,std,four,...,55.5,2952,ohc,four,141,mpfi,9.5,23,28,16845
196,95.0,3.78,3.15,160.0,5300.0,-1,volvo,gas,turbo,four,...,55.5,3049,ohc,four,141,mpfi,8.7,19,25,19045
197,95.0,3.58,2.87,134.0,5500.0,-1,volvo,gas,std,four,...,55.5,3012,ohcv,six,173,mpfi,8.8,18,23,21485
198,95.0,3.01,3.40,106.0,4800.0,-1,volvo,diesel,turbo,four,...,55.5,3217,ohc,six,145,idi,23.0,26,27,22470


### (i)

In [547]:
for col in ['num_doors', 'num_cylinders']:
    for i in range(0, len(df[col])):
        word=df.at[i, col]
        if word=='two':
            df.at[i, col]=2
        elif word=='three':
            df.at[i, col]=3
        elif word=='four':
            df.at[i, col]=4
        elif word=='five':
            df.at[i, col]=5
        elif word=='six':
            df.at[i, col]=6
        elif word=='eight':
            df.at[i, col]=8
        elif word=='twelve':
            df.at[i, col]=12


In [548]:
df1['num_doors']=pd.DataFrame((mean_imputer.fit_transform((df['num_doors'].values).reshape(-1, 1))).flatten())

for col in df.columns:
    if col not in ['num_doors']:
        df1[col]=df[col]

df=df1
df=df.dropna(subset=['price'])
df

,normalized_losses,bore,stroke,horsepower,peak_rpm,symboling,make,fuel_type,aspiration,num_doors,...,height,curb_weight,engine_type,num_cylinders,engine_size,fuel_system,compression_ratio,city_mpg,highway_mpg,price
0,122.0,3.47,2.68,111.0,5000.0,3.0,alfa-romero,gas,std,2.0,...,48.8,2548.0,dohc,4,130.0,mpfi,9.0,21.0,27.0,16500
1,122.0,2.68,3.47,154.0,5000.0,1.0,alfa-romero,gas,std,2.0,...,52.4,2823.0,ohcv,6,152.0,mpfi,9.0,19.0,26.0,16500
2,164.0,3.19,3.40,102.0,5500.0,2.0,audi,gas,std,4.0,...,54.3,2337.0,ohc,4,109.0,mpfi,10.0,24.0,30.0,13950
3,164.0,3.19,3.40,115.0,5500.0,2.0,audi,gas,std,4.0,...,54.3,2824.0,ohc,5,136.0,mpfi,8.0,18.0,22.0,17450
4,122.0,3.19,3.40,110.0,5500.0,2.0,audi,gas,std,2.0,...,53.1,2507.0,ohc,5,136.0,mpfi,8.5,19.0,25.0,15250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,95.0,3.78,3.15,114.0,5400.0,-1.0,volvo,gas,std,4.0,...,55.5,2952.0,ohc,4,141.0,mpfi,9.5,23.0,28.0,16845
196,95.0,3.78,3.15,160.0,5300.0,-1.0,volvo,gas,turbo,4.0,...,55.5,3049.0,ohc,4,141.0,mpfi,8.7,19.0,25.0,19045
197,95.0,3.58,2.87,134.0,5500.0,-1.0,volvo,gas,std,4.0,...,55.5,3012.0,ohcv,6,173.0,mpfi,8.8,18.0,23.0,21485
198,95.0,3.01,3.40,106.0,4800.0,-1.0,volvo,diesel,turbo,4.0,...,55.5,3217.0,ohc,6,145.0,idi,23.0,26.0,27.0,22470


### (ii)

In [549]:
print(df['body_style'].unique())
print(df['drive_wheels'].unique())

['convertible' 'hatchback' 'sedan' 'wagon' 'hardtop']
['rwd' 'fwd' '4wd']


In [550]:
encoder = OneHotEncoder()

encoder_df = pd.DataFrame(encoder.fit_transform(df[['body_style']]).toarray())
df=df.join(encoder_df)
df.rename(columns = {0:'convertible', 1:'hatchback', 2:'sedan', 3:'wagon', 4:'hardtop'}, inplace = True)
df=df.drop('body_style', axis=1)
df=df.drop('convertible', axis=1)
df

,normalized_losses,bore,stroke,horsepower,peak_rpm,symboling,make,fuel_type,aspiration,num_doors,...,engine_size,fuel_system,compression_ratio,city_mpg,highway_mpg,price,hatchback,sedan,wagon,hardtop
0,122.0,3.47,2.68,111.0,5000.0,3.0,alfa-romero,gas,std,2.0,...,130.0,mpfi,9.0,21.0,27.0,16500,0.0,0.0,0.0,0.0
1,122.0,2.68,3.47,154.0,5000.0,1.0,alfa-romero,gas,std,2.0,...,152.0,mpfi,9.0,19.0,26.0,16500,0.0,1.0,0.0,0.0
2,164.0,3.19,3.40,102.0,5500.0,2.0,audi,gas,std,4.0,...,109.0,mpfi,10.0,24.0,30.0,13950,0.0,0.0,1.0,0.0
3,164.0,3.19,3.40,115.0,5500.0,2.0,audi,gas,std,4.0,...,136.0,mpfi,8.0,18.0,22.0,17450,0.0,0.0,1.0,0.0
4,122.0,3.19,3.40,110.0,5500.0,2.0,audi,gas,std,2.0,...,136.0,mpfi,8.5,19.0,25.0,15250,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,95.0,3.78,3.15,114.0,5400.0,-1.0,volvo,gas,std,4.0,...,141.0,mpfi,9.5,23.0,28.0,16845,0.0,0.0,1.0,0.0
196,95.0,3.78,3.15,160.0,5300.0,-1.0,volvo,gas,turbo,4.0,...,141.0,mpfi,8.7,19.0,25.0,19045,0.0,0.0,1.0,0.0
197,95.0,3.58,2.87,134.0,5500.0,-1.0,volvo,gas,std,4.0,...,173.0,mpfi,8.8,18.0,23.0,21485,0.0,0.0,1.0,0.0
198,95.0,3.01,3.40,106.0,4800.0,-1.0,volvo,diesel,turbo,4.0,...,145.0,idi,23.0,26.0,27.0,22470,0.0,0.0,1.0,0.0


In [551]:
encoder = OneHotEncoder()

encoder_df = pd.DataFrame(encoder.fit_transform(df[['drive_wheels']]).toarray())
df=df.join(encoder_df)
df.rename(columns = {0:'rwd', 1:'fwd', 2:'4wd'}, inplace = True)
df=df.drop('drive_wheels', axis=1)
df=df.drop('rwd', axis=1)
df

,normalized_losses,bore,stroke,horsepower,peak_rpm,symboling,make,fuel_type,aspiration,num_doors,...,compression_ratio,city_mpg,highway_mpg,price,hatchback,sedan,wagon,hardtop,fwd,4wd
0,122.0,3.47,2.68,111.0,5000.0,3.0,alfa-romero,gas,std,2.0,...,9.0,21.0,27.0,16500,0.0,0.0,0.0,0.0,0.0,1.0
1,122.0,2.68,3.47,154.0,5000.0,1.0,alfa-romero,gas,std,2.0,...,9.0,19.0,26.0,16500,0.0,1.0,0.0,0.0,0.0,1.0
2,164.0,3.19,3.40,102.0,5500.0,2.0,audi,gas,std,4.0,...,10.0,24.0,30.0,13950,0.0,0.0,1.0,0.0,1.0,0.0
3,164.0,3.19,3.40,115.0,5500.0,2.0,audi,gas,std,4.0,...,8.0,18.0,22.0,17450,0.0,0.0,1.0,0.0,0.0,0.0
4,122.0,3.19,3.40,110.0,5500.0,2.0,audi,gas,std,2.0,...,8.5,19.0,25.0,15250,0.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,95.0,3.78,3.15,114.0,5400.0,-1.0,volvo,gas,std,4.0,...,9.5,23.0,28.0,16845,0.0,0.0,1.0,0.0,0.0,1.0
196,95.0,3.78,3.15,160.0,5300.0,-1.0,volvo,gas,turbo,4.0,...,8.7,19.0,25.0,19045,0.0,0.0,1.0,0.0,0.0,1.0
197,95.0,3.58,2.87,134.0,5500.0,-1.0,volvo,gas,std,4.0,...,8.8,18.0,23.0,21485,0.0,0.0,1.0,0.0,0.0,1.0
198,95.0,3.01,3.40,106.0,4800.0,-1.0,volvo,diesel,turbo,4.0,...,23.0,26.0,27.0,22470,0.0,0.0,1.0,0.0,0.0,1.0


### (iii)

In [552]:
le = LabelEncoder()
df['make']=le.fit_transform(df['make'])
df['fuel_type']=le.fit_transform(df['fuel_type'])
df['aspiration']=le.fit_transform(df['aspiration'])
df['engine_location']=le.fit_transform(df['engine_location'])
df


,normalized_losses,bore,stroke,horsepower,peak_rpm,symboling,make,fuel_type,aspiration,num_doors,...,compression_ratio,city_mpg,highway_mpg,price,hatchback,sedan,wagon,hardtop,fwd,4wd
0,122.0,3.47,2.68,111.0,5000.0,3.0,0,1,0,2.0,...,9.0,21.0,27.0,16500,0.0,0.0,0.0,0.0,0.0,1.0
1,122.0,2.68,3.47,154.0,5000.0,1.0,0,1,0,2.0,...,9.0,19.0,26.0,16500,0.0,1.0,0.0,0.0,0.0,1.0
2,164.0,3.19,3.40,102.0,5500.0,2.0,1,1,0,4.0,...,10.0,24.0,30.0,13950,0.0,0.0,1.0,0.0,1.0,0.0
3,164.0,3.19,3.40,115.0,5500.0,2.0,1,1,0,4.0,...,8.0,18.0,22.0,17450,0.0,0.0,1.0,0.0,0.0,0.0
4,122.0,3.19,3.40,110.0,5500.0,2.0,1,1,0,2.0,...,8.5,19.0,25.0,15250,0.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,95.0,3.78,3.15,114.0,5400.0,-1.0,21,1,0,4.0,...,9.5,23.0,28.0,16845,0.0,0.0,1.0,0.0,0.0,1.0
196,95.0,3.78,3.15,160.0,5300.0,-1.0,21,1,1,4.0,...,8.7,19.0,25.0,19045,0.0,0.0,1.0,0.0,0.0,1.0
197,95.0,3.58,2.87,134.0,5500.0,-1.0,21,1,0,4.0,...,8.8,18.0,23.0,21485,0.0,0.0,1.0,0.0,0.0,1.0
198,95.0,3.01,3.40,106.0,4800.0,-1.0,21,0,1,4.0,...,23.0,26.0,27.0,22470,0.0,0.0,1.0,0.0,0.0,1.0


### (iv)

In [553]:
for i in df.index:
    if(df.at[i, 'fuel_system']=='pfi'):
        df.at[i, 'fuel_system']=1
    else:
        df.at[i, 'fuel_system']=0

df['fuel_system']

0      0
1      0
2      0
3      0
4      0
      ..
195    0
196    0
197    0
198    0
199    0
Name: fuel_system, Length: 200, dtype: object

### (v)

In [554]:
for i in df.index:
    if(df.at[i, 'engine_type']=='ohc'):
        df.at[i, 'engine_type']=1
    else:
        df.at[i, 'engine_type']=0

df['engine_type']

0      0
1      0
2      1
3      1
4      1
      ..
195    1
196    1
197    0
198    1
199    1
Name: engine_type, Length: 200, dtype: object

## 4.

In [555]:
X=df.iloc[:, 0:23].copy()
X=X.join(df.iloc[:, 24:30].copy())
X

,normalized_losses,bore,stroke,horsepower,peak_rpm,symboling,make,fuel_type,aspiration,num_doors,...,fuel_system,compression_ratio,city_mpg,highway_mpg,hatchback,sedan,wagon,hardtop,fwd,4wd
0,122.0,3.47,2.68,111.0,5000.0,3.0,0,1,0,2.0,...,0,9.0,21.0,27.0,0.0,0.0,0.0,0.0,0.0,1.0
1,122.0,2.68,3.47,154.0,5000.0,1.0,0,1,0,2.0,...,0,9.0,19.0,26.0,0.0,1.0,0.0,0.0,0.0,1.0
2,164.0,3.19,3.40,102.0,5500.0,2.0,1,1,0,4.0,...,0,10.0,24.0,30.0,0.0,0.0,1.0,0.0,1.0,0.0
3,164.0,3.19,3.40,115.0,5500.0,2.0,1,1,0,4.0,...,0,8.0,18.0,22.0,0.0,0.0,1.0,0.0,0.0,0.0
4,122.0,3.19,3.40,110.0,5500.0,2.0,1,1,0,2.0,...,0,8.5,19.0,25.0,0.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,95.0,3.78,3.15,114.0,5400.0,-1.0,21,1,0,4.0,...,0,9.5,23.0,28.0,0.0,0.0,1.0,0.0,0.0,1.0
196,95.0,3.78,3.15,160.0,5300.0,-1.0,21,1,1,4.0,...,0,8.7,19.0,25.0,0.0,0.0,1.0,0.0,0.0,1.0
197,95.0,3.58,2.87,134.0,5500.0,-1.0,21,1,0,4.0,...,0,8.8,18.0,23.0,0.0,0.0,1.0,0.0,0.0,1.0
198,95.0,3.01,3.40,106.0,4800.0,-1.0,21,0,1,4.0,...,0,23.0,26.0,27.0,0.0,0.0,1.0,0.0,0.0,1.0


In [556]:
Y=df.iloc[:, 23].copy()
Y

0      16500
1      16500
2      13950
3      17450
4      15250
       ...  
195    16845
196    19045
197    21485
198    22470
199    22625
Name: price, Length: 200, dtype: object

In [557]:
ss=StandardScaler()
Xstd=ss.fit_transform(X)

for i in range(0, len(X.columns)):
    X[X.columns[i]]=Xstd[:, i]

X

,normalized_losses,bore,stroke,horsepower,peak_rpm,symboling,make,fuel_type,aspiration,num_doors,...,fuel_system,compression_ratio,city_mpg,highway_mpg,hatchback,sedan,wagon,hardtop,fwd,4wd
0,0.000000,0.522674,-1.850140,0.204293,-0.247384,1.742368,-1.996136,0.333333,-0.468521,-1.158810,...,0.0,-0.292224,-0.654574,-0.544042,-0.204124,-0.717741,-0.941697,-0.377964,-1.199593,1.304877
1,0.000000,-2.426299,0.670678,1.355201,-0.247384,0.136499,-1.996136,0.333333,-0.468521,-1.158810,...,0.0,-0.292224,-0.966276,-0.690882,-0.204124,1.393261,-0.941697,-0.377964,-1.199593,1.304877
2,1.312654,-0.522532,0.447314,-0.036594,0.798551,0.939433,-1.834636,0.333333,-0.468521,0.871671,...,0.0,-0.042481,-0.187021,-0.103522,-0.204124,-0.717741,1.061913,-0.377964,0.833616,-0.766356
3,1.312654,-0.522532,0.447314,0.311355,0.798551,0.939433,-1.834636,0.333333,-0.468521,0.871671,...,0.0,-0.541967,-1.122127,-1.278241,-0.204124,-0.717741,1.061913,-0.377964,-1.199593,-0.766356
4,0.000000,-0.522532,0.447314,0.177528,0.798551,0.939433,-1.834636,0.333333,-0.468521,-1.158810,...,0.0,-0.417096,-0.966276,-0.837722,-0.204124,-0.717741,1.061913,-0.377964,0.833616,-0.766356
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,-0.843849,1.679865,-0.350413,0.284589,0.589364,-1.469370,1.395357,0.333333,-0.468521,0.871671,...,0.0,-0.167353,-0.342872,-0.397202,-0.204124,-0.717741,1.061913,-0.377964,-1.199593,1.304877
196,-0.843849,1.679865,-0.350413,1.515793,0.380177,-1.469370,1.395357,0.333333,2.134375,0.871671,...,0.0,-0.367147,-0.966276,-0.837722,-0.204124,-0.717741,1.061913,-0.377964,-1.199593,1.304877
197,-0.843849,0.933290,-1.243867,0.819895,0.798551,-1.469370,1.395357,0.333333,-0.468521,0.871671,...,0.0,-0.342173,-1.122127,-1.131402,-0.204124,-0.717741,1.061913,-0.377964,-1.199593,1.304877
198,-0.843849,-1.194450,0.447314,0.070467,-0.665758,-1.469370,1.395357,-3.000000,2.134375,0.871671,...,0.0,3.204179,0.124681,-0.544042,-0.204124,-0.717741,1.061913,-0.377964,-1.199593,1.304877


## 5.

In [558]:
X_train,X_test,Y_train,Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

model = LinearRegression()
model.fit(X_train,Y_train)
print(f"Before PCA:\tScore={model.score(X_test, Y_test)}")

Before PCA:	Score=0.7613674862209823


In [559]:
pca = PCA(0.95)
X_pca = pca.fit_transform(X)

X_train_pca, X_test_pca, Y_train, Y_test = train_test_split(X_pca, Y, test_size=0.3)

model = LinearRegression()
model.fit(X_train_pca, Y_train)
print(f"After PCA:\tScore={model.score(X_test_pca, Y_test)}")

After PCA:	Score=0.8432696801818383
